#Importing libraries


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import sklearn

In [ ]:
sklearn.__version__

'0.22.2.post1'

#Loading dataset

In [ ]:
dataset=pd.read_csv("heart_failure_clinical_records_dataset.csv")
X=dataset.iloc[:,:-1].values
y=dataset.iloc[:,-1].values

In [ ]:
dataset.shape

(299, 13)

#Data preprocessing


#Feature Selection using Correlation

In [ ]:
corr_matrix = dataset.corr()
corr_matrix

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
age,1.000000,0.088006,-0.081584,-0.101012,0.060098,0.093289,-0.052354,0.159187,-0.045966,0.065430,0.018668,-0.224068,0.253729
anaemia,0.088006,1.000000,-0.190741,-0.012729,0.031557,0.038182,-0.043786,0.052174,0.041882,-0.094769,-0.107290,-0.141414,0.066270
creatinine_phosphokinase,-0.081584,-0.190741,1.000000,-0.009639,-0.044080,-0.070590,0.024463,-0.016408,0.059550,0.079791,0.002421,-0.009346,0.062728
diabetes,-0.101012,-0.012729,-0.009639,1.000000,-0.004850,-0.012732,0.092193,-0.046975,-0.089551,-0.157730,-0.147173,0.033726,-0.001943
ejection_fraction,0.060098,0.031557,-0.044080,-0.004850,1.000000,0.024445,0.072177,-0.011302,0.175902,-0.148386,-0.067315,0.041729,-0.268603
high_blood_pressure,0.093289,0.038182,-0.070590,-0.012732,0.024445,1.000000,0.049963,-0.004935,0.037109,-0.104615,-0.055711,-0.196439,0.079351
platelets,-0.052354,-0.043786,0.024463,0.092193,0.072177,0.049963,1.000000,-0.041198,0.062125,-0.125120,0.028234,0.010514,-0.049139
serum_creatinine,0.159187,0.052174,-0.016408,-0.046975,-0.011302,-0.004935,-0.041198,1.000000,-0.189095,0.006970,-0.027414,-0.149315,0.294278
serum_sodium,-0.045966,0.041882,0.059550,-0.089551,0.175902,0.037109,0.062125,-0.189095,1.000000,-0.027566,0.004813,0.087640,-0.195204
sex,0.065430,-0.094769,0.079791,-0.157730,-0.148386,-0.104615,-0.125120,0.006970,-0.027566,1.000000,0.445892,-0.015608,-0.004316


In [ ]:
def get_correlation(df, threshold):
  corr_matrix = df.to_numpy()
  columns = df.columns
  col = set()
  for i in range(len(corr_matrix)):
    for j in range(i+1,len(corr_matrix)):
      if(abs(corr_matrix[i][j]) > threshold ):
        col.add(columns[j])
 
  return col

In [ ]:
print(get_correlation(corr_matrix,0.85))

set()


#Splitting the data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

#Outlier Detection

In [ ]:
from sklearn.ensemble import IsolationForest
detector = IsolationForest(n_estimators=100, contamination=0.05)
anamoly_rows = detector.fit_predict(X_train)

In [ ]:
anamoly_rows

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,
        1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1, -1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,
        1,  1,  1,  1,  1

In [ ]:
ctr=0
for i in anamoly_rows:
  if(i==-1):
    ctr+=1
print("Outlier rows found are : ", ctr)

Outlier rows found are :  12


In [ ]:
m = anamoly_rows !=-1
X_train  = X_train[m!=False,:]
y_train = y_train[m!=False]

In [ ]:
print(X_train.shape)
print(y_train.shape)

(227, 12)
(227,)


#Training (unrestricted)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

#Testing unregularized Model

In [ ]:
y_pred = classifier.predict(X_test)
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
print("Accuracy: ",accuracy_score(y_test, y_pred) )

[[37  0]
 [12 11]]
Accuracy:  0.8


#K-fold Cross validation (unrestricted)

In [ ]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 79.78 %
Standard Deviation: 6.98 %


#Tuning the hyperparameters

In [ ]:
parameters=[{'n_estimators':[5,7,10,12,15,17,20,23,25,30,35,40,45,50,55,60,65],'max_features' : ['auto', 'sqrt'],
           'max_depth':[5,7,10,12,15,17,20,23,25],'bootstrap': [True, False],
           'min_samples_leaf': [1, 2, 4], 'min_samples_split': [2, 5, 10] ,'criterion':['ginni','entropy'] } ]

from sklearn.model_selection import GridSearchCV
search=GridSearchCV(estimator=classifier,cv=10,n_jobs=-1,scoring='accuracy',param_grid=parameters)
search.fit(X_train,y_train)

best_acc=search.best_score_
best_param=search.best_params_

In [ ]:
print("best accuracy :",best_acc)
print("Shortlisted parameters:",best_param)


best accuracy : 0.8810276679841896
Shortlisted parameters: {'bootstrap': True, 'criterion': 'entropy', 'max_depth': 7, 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 25}


#Using the shortlisted parameters for the model 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(bootstrap=True, max_depth=7, max_features='auto',
                                    min_samples_leaf= 2,min_samples_split= 5,n_estimators=25, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=7, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=25,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

#Cross validation

In [ ]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 88.10 %
Standard Deviation: 3.93 %


In [ ]:
print("Evaluation accuracy:",accuracies)

Evaluation accuracy: [0.82608696 0.86956522 0.86956522 0.95652174 0.82608696 0.91304348
 0.91304348 0.90909091 0.86363636 0.86363636]


#Making of confusion Matrix 

In [ ]:
y_pred = classifier.predict(X_test)
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
print("Accuracy: {:.2f}".format(accuracy_score(y_test, y_pred) ) )

[[35  2]
 [ 9 14]]
Accuracy: 0.82


#Saving the model

In [ ]:
import joblib
file_name='model_save'
joblib.dump(classifier,file_name)

['model_save']

#Loading the model

In [ ]:
import joblib
model=joblib.load('/content/model_save')

In [ ]:
y_p=model.predict(X_test)

#Accuracy of the loaded Model

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_p)

[[36  1]
 [ 7 16]]


0.8666666666666667